## Testing out parenx skeletonization and voronoi approaches

Resources:
* https://github.com/nptscot/networkmerge
* https://github.com/nptscot/networkmerge
* https://github.com/anisotropi4/parenx/tree/main

In [ ]:
import parenx
import glob
import os
import geopandas as gpd

In [11]:
# parquet is not a recognized format - convert to gpkg first
os.makedirs("../temp-parenx/", exist_ok=True)
folders = os.listdir("../data/")
folders.remove("sample.parquet")
for folder in folders:
    os.makedirs(f"../temp-parenx/{folder}/", exist_ok=True)
    if not folder.endswith("parquet"):
        roads = gpd.read_parquet(f"../data/{folder}/roads_osm.parquet").reset_index(drop=True)
        roads.to_file(
            f"../temp-parenx/{folder}/roads_osm.gpkg", 
            layer="roads", 
            engine="pyogrio"
        )

869
1656
4881
809
1133
4617


**Now, run the bash script `parenx-run.sh` from command line; will add to each subfolder in `temp-parenx` 2 files: voronoi.gpkg and skeletonization.gpkg**